In [1]:
import pandas as pd
import numpy as np

In [2]:
popularity = pd.read_csv('Meal Plan Data - Dining Hall Popularity.csv')
donations = pd.read_csv('Meal Plan Data - Donations.csv')
location = pd.read_csv('Meal Plan Data - Hill vs Campus.csv')
enrollment = pd.read_csv('Meal Plan Data - Enrollment.csv')
leftover = pd.read_csv('Meal Plan Data - Leftover.csv')

In [3]:
popularity

,Location,Weekly Swipes
0,Epicuria at Covel,8998
1,De Neve,22227
2,Spice Kitchen at Feast,7413
3,Bruin Plate,24999
4,Bruin Café,9983
5,Café 1919,4057
6,Rendezvous,21299
7,The Study at Hedrick,29589
8,The Drey,5023


In [8]:
hill_total = popularity.iloc[:,1].sum()
location_total = (hill_total/0.8).astype(int)
campus_total = (0.2*location_total).astype(int)
print(hill_total, campus_total, location_total)


133588 33397 166985


In [14]:
percentages = []
for i in popularity.iloc[:,1]:
    a = i/location_total
    percentages.append(a)
percentages.append(campus_total/location_total)
percentages

[0.05388507949815852,
 0.13310776417043446,
 0.04439320897086565,
 0.14970805760996497,
 0.05978381291732791,
 0.024295595412761627,
 0.12755037877653683,
 0.17719555648710963,
 0.030080546156840436,
 0.2]

In [4]:
donations

,Quarter,Swipes
0,Fall 2022,32684
1,Winter 2023,18235
2,Spring 2023,16754


In [5]:
location

,Quarter,The Hill,On Campus
0,F23,0.8,0.2


In [6]:
enrollment

,Unnamed: 0,11P,11R,14P,14R,19P,19R
0,2223,1491,444,5241,951,3461,1670
1,2324,1530,371,5814,938,4199,1584


In [7]:
leftover

,Unnamed: 0,11P,11R,14P,14R,19P,19R
0,F22,23486,1852,76959,6712,85095,16070
1,W23,19256,1870,49481,6351,57916,15111
2,S23,18531,1708,50659,5678,56292,14436


Stacked bar chart with enrollment to count consumption, leftovers, and donations. 

In [8]:
enroll = enrollment.drop('Unnamed: 0', axis = 1)
for col in enroll.columns:
    if '11' in col:
        enroll[col] *= 11*34
    elif '14' in col:
        enroll[col] *= 14*34
    elif '19' in col:
        enroll[col] *= 19*34
print(enroll)

      11P     11R      14P     14R      19P      19R
0  557634  166056  2494716  452676  2235806  1078820
1  572220  138754  2767464  446488  2712554  1023264


In [9]:
#find proportion of enrollment for each meal plan relative to total enrollment
e_colsum = enroll.iloc[0,:]
totalsum = e_colsum.sum()
e_ratios = e_colsum/totalsum
print(e_colsum, e_ratios)

11P     557634
11R     166056
14P    2494716
14R     452676
19P    2235806
19R    1078820
Name: 0, dtype: int64 11P    0.079825
11R    0.023771
14P    0.357117
14R    0.064800
19P    0.320054
19R    0.154432
Name: 0, dtype: float64


In [10]:
#find proportion of leftovers for each meal plan relative to total leftovers
l_colsum = leftover.iloc[:,1:].sum()
totalsum = l_colsum.sum()
l_ratios = l_colsum/totalsum
print(l_colsum, l_ratios)

11P     61273
11R      5430
14P    177099
14R     18741
19P    199303
19R     45617
dtype: int64 11P    0.120744
11R    0.010700
14P    0.348989
14R    0.036931
19P    0.392744
19R    0.089892
dtype: float64


In [32]:
donation = donations.iloc[:,1].sum()
data = {
    '11P': [(l_ratios[0]*donation).astype(int)],
    '11R': [(l_ratios[1]*donation).astype(int)],
    '14P': [(l_ratios[2]*donation).astype(int)],
    '14R': [(l_ratios[3]*donation).astype(int)],
    '19P': [(l_ratios[4]*donation).astype(int)],
    '19R': [(l_ratios[5]*donation).astype(int)],
}

donations_est = pd.DataFrame(data)
donations_est = donations_est.iloc[0,:]
donations_est

11P     8171
11R      724
14P    23617
14R     2499
19P    26578
19R     6083
Name: 0, dtype: int64

In [33]:
consumed = e_colsum - l_colsum

In [35]:
print(e_colsum)
print(consumed)
print(l_colsum)
print(donations_est)

11P     557634
11R     166056
14P    2494716
14R     452676
19P    2235806
19R    1078820
Name: 0, dtype: int64
11P     496361
11R     160626
14P    2317617
14R     433935
19P    2036503
19R    1033203
dtype: int64
11P     61273
11R      5430
14P    177099
14R     18741
19P    199303
19R     45617
dtype: int64
11P     8171
11R      724
14P    23617
14R     2499
19P    26578
19R     6083
Name: 0, dtype: int64


In [46]:
total = pd.concat([consumed, l_colsum, donations_est], axis=1)
total

,0,1,0
11P,496361,61273,8171
11R,160626,5430,724
14P,2317617,177099,23617
14R,433935,18741,2499
19P,2036503,199303,26578
19R,1033203,45617,6083


In [55]:
plans = ['11P', '11R', '14P', '14R', '19P', '19R']

print("{\n \"name\": \"Meal Plan\", \n \"children\": [ \n")
for i in range(6):
    print("{ \n \"name\": \"", plans[i], "\" \n \"children\": ["),
    print("{\"name\": \"Consumed\", \"size\":", total.iloc[i,0], "}," )
    print("{\"name\": \"Leftover\", \"size\":", total.iloc[i,1], "}," )
    print("{\"name\": \"Donated\", \"size\":", total.iloc[i,2], "} \n ] \n }," )
print("] \n }")

{
    "name": "Meal Plan",
    "children": [
        {
            "name": "11P",
            "children": [
                { "name": "Consumed", "size": 496361 },
                { "name": "Leftovers", "size": 53101 },
                { "name": "Donations", "size": 8171 }
            ]
        },
        {
            "name": "11R",
            "children": [
                { "name": "Consumed", "size": 496361 },
                { "name": "Leftovers", "size": 53101 },
                { "name": "Donations", "size": 8171 }
            ]
        },
        {
            "name": "14P",
            "children": [
                { "name": "Consumed", "size": 496361 },
                { "name": "Leftovers", "size": 53101 },
                { "name": "Donations", "size": 8171 }
            ]
        },
        {
            "name": "14R",
            "children": [
                { "name": "Consumed", "size": 496361 },
                { "name": "Leftovers", "size": 53101 },
                { "name": "Donations", "size": 8171 }
            ]
        },
        {
            "name": "19P",
            "children": [
                { "name": "Consumed", "size": 496361 },
                { "name": "Leftovers", "size": 53101 },
                { "name": "Donations", "size": 8171 }
            ]
        },
        {
            "name": "19R",
            "children": [
                { "name": "Consumed", "size": 496361 },
                { "name": "Leftovers", "size": 53101 },
                { "name": "Donations", "size": 8171 }
            ]
        }
    ]
}


{
 "name": "Meal Plan", 
 "children": [ 

{ 
 "name": " 11P " 
 "children": [
{"name": "Consumed", "size": 496361 },
{"name": "Leftover", "size": 61273 },
{"name": "Donated", "size": 8171 } 
 ] 
 },
{ 
 "name": " 11R " 
 "children": [
{"name": "Consumed", "size": 160626 },
{"name": "Leftover", "size": 5430 },
{"name": "Donated", "size": 724 } 
 ] 
 },
{ 
 "name": " 14P " 
 "children": [
{"name": "Consumed", "size": 2317617 },
{"name": "Leftover", "size": 177099 },
{"name": "Donated", "size": 23617 } 
 ] 
 },
{ 
 "name": " 14R " 
 "children": [
{"name": "Consumed", "size": 433935 },
{"name": "Leftover", "size": 18741 },
{"name": "Donated", "size": 2499 } 
 ] 
 },
{ 
 "name": " 19P " 
 "children": [
{"name": "Consumed", "size": 2036503 },
{"name": "Leftover", "size": 199303 },
{"name": "Donated", "size": 26578 } 
 ] 
 },
{ 
 "name": " 19R " 
 "children": [
{"name": "Consumed", "size": 1033203 },
{"name": "Leftover", "size": 45617 },
{"name": "Donated", "size": 6083 } 
 ] 
 },
] 
 }


{'name': 'Meal Plan',
 'children': [{'name': '11P',
   'children': [{'name': 'Consumed', 'size': 496361},
    {'name': 'Leftovers', 'size': 53101},
    {'name': 'Donations', 'size': 8171}]},
  {'name': '11R',
   'children': [{'name': 'Consumed', 'size': 496361},
    {'name': 'Leftovers', 'size': 53101},
    {'name': 'Donations', 'size': 8171}]},
  {'name': '14P',
   'children': [{'name': 'Consumed', 'size': 496361},
    {'name': 'Leftovers', 'size': 53101},
    {'name': 'Donations', 'size': 8171}]},
  {'name': '14R',
   'children': [{'name': 'Consumed', 'size': 496361},
    {'name': 'Leftovers', 'size': 53101},
    {'name': 'Donations', 'size': 8171}]},
  {'name': '19P',
   'children': [{'name': 'Consumed', 'size': 496361},
    {'name': 'Leftovers', 'size': 53101},
    {'name': 'Donations', 'size': 8171}]},
  {'name': '19R',
   'children': [{'name': 'Consumed', 'size': 496361},
    {'name': 'Leftovers', 'size': 53101},
    {'name': 'Donations', 'size': 8171}]}]}